In [923]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [924]:
import pandas as pd

from functions import *

pd.set_option('display.max_columns', None)

df_merged = pd.read_csv("data/df_merged_national.csv")

In [925]:
df_merged = clean_df(df_merged)
df_merged = every_float_to_int(df_merged)

In [926]:
df_no_provincias = df_merged[(df_merged["region"] == "    DESCONOCIDA") | (df_merged["region"] == "    EN EL EXTRANJERO") | (df_merged["region"] == "    TOTAL NACIONAL")]

df_no_provincias.to_csv("data/df_no_provincias.csv", index=False)

In [927]:
df_merged = df_merged[~df_merged.index.isin(df_no_provincias.index)]

In [928]:
df_num = df_merged.iloc[:, df_merged.columns != 'region']

In [929]:
df_merged = add_coordinates_from_dict(df_merged)

In [930]:
regions = {
    "ANDALUCÍA": 0,
    "ARAGÓN": 1,
    "ASTURIAS (PRINCIPADO DE)": 2,
    "BALEARS (ILLES)": 3,
    "CANTABRIA": 4,
    "CANARIAS": 5,
    "CASTILLA - LA MANCHA": 6,
    "CASTILLA Y LEÓN": 7,
    "CATALUÑA": 8,
    "CIUDAD AUTÓNOMA DE CEUTA": 9,
    "CIUDAD AUTÓNOMA DE MELILLA": 10,
    "COMUNITAT VALENCIANA": 11,
    "EXTREMADURA": 12,
    "GALICIA": 13,
    "MADRID (COMUNIDAD DE)": 14,
    "MURCIA (REGIÓN DE)": 15,
    "NAVARRA (COMUNIDAD FORAL DE)": 16,
    "PAÍS VASCO": 17,
    "RIOJA (LA)": 18
}


df_merged["region"] = df_merged["region"].replace(regions)

C:\Users\Iván\AppData\Local\Temp\ipykernel_1992\291359535.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_merged["region"] = df_merged["region"].replace(regions)


In [931]:
df_merged.insert(0, "year", df_merged.pop("year"))

In [932]:
df_merged["avg_crimes"] = df_merged.iloc[:, 2:-4].mean(axis=1)      # [:, 2:-2] quiero -> todas las filas, desde la 2da hasta la penultima columna

In [933]:
df_merged = delete_sub_crimes(df_merged)

In [934]:
df_merged["years_from_today"] = 2024 - df_merged["year"]

In [935]:
df_poblacion = pd.read_csv("data/cleaned_pop.csv")


df_poblacion.drop(["Nacional", "Sexo"], inplace=True, axis=1)

regions_pop_version = {
    "01 Andalucía": 0,
    "02 Aragón": 1,
    "03 Asturias Principado de": 2,
    "04 Balears Illes": 3,
    "06 Cantabria": 4,
    "05 Canarias": 5,
    "08 Castilla - La Mancha": 6,
    "07 Castilla y León": 7,
    "09 Cataluña": 8,
    "18 Ceuta": 9,
    "19 Melilla": 10,
    "10 Comunitat Valenciana": 11,
    "11 Extremadura": 12,
    "12 Galicia": 13,
    "13 Madrid Comunidad de": 14,
    "14 Murcia Región de": 15,
    "15 Navarra Comunidad Foral de": 16,
    "16 País Vasco": 17,
    "17 Rioja La": 18
}


df_poblacion["region"] = df_poblacion["region"].replace(regions_pop_version)

C:\Users\Iván\AppData\Local\Temp\ipykernel_1992\1471067258.py:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_poblacion["region"] = df_poblacion["region"].replace(regions_pop_version)


In [936]:
df_poblacion

,region,year,Total
0,0,2019,8.414.240
1,0,2018,8.384.408
2,0,2017,8.379.820
3,0,2016,8.388.107
4,0,2015,8.399.043
...,...,...,...
185,10,2014,84.509
186,10,2013,83.679
187,10,2012,80.802
188,10,2011,78.476


In [937]:
df_merged = df_merged.merge(df_poblacion, on=["region", "year"])

df_merged.rename(columns={"Total": "population"}, inplace=True)

In [938]:
df_merged.columns

Index(['year', 'region', '1_against_people', '2_against_freedom',
       '3_sexual_freedom', '4_family_relationships', '5_property_crimes',
       '6_collective_security', '7_forgeries', '8_public_administration',
       '9_justice_administration', 'latitude', 'longitude', 'avg_crimes',
       'years_from_today', 'population'],
      dtype='object')

In [939]:
crime_values = ['1_against_people', '2_against_freedom',
       '3_sexual_freedom', '4_family_relationships', '5_property_crimes',
       '6_collective_security', '7_forgeries', '8_public_administration',
       '9_justice_administration']

In [940]:
df_unpivot = pd.melt(df_merged, id_vars=['year', 'region', 'latitude', 'longitude', 'population'], value_vars=crime_values)

df_unpivot

,year,region,latitude,longitude,population,variable,value
0,2019,0,37.3873,-5.9869,8.414.240,1_against_people,37130
1,2019,1,41.6488,-0.8891,1.319.291,1_against_people,4760
2,2019,2,43.3619,-5.8494,1.022.800,1_against_people,3368
3,2019,3,39.5712,2.6466,1.149.460,1_against_people,8110
4,2019,5,28.2916,-16.6291,2.153.389,1_against_people,12396
...,...,...,...,...,...,...,...
1705,2010,16,42.6954,-1.6761,636.924,9_justice_administration,235
1706,2010,17,43.2630,-2.9349,2.178.339,9_justice_administration,1170
1707,2010,18,42.2871,-2.5396,322.415,9_justice_administration,137
1708,2010,9,35.8894,-5.3198,80.579,9_justice_administration,101


In [941]:
df_unpivot_transformed = df_unpivot.copy()

for crime in crime_values:
    column_name = f'{crime}_indicator'
    df_unpivot_transformed[column_name] = (df_unpivot_transformed['variable'] == crime).astype(int)

df_unpivot_transformed

,year,region,latitude,longitude,population,variable,value,1_against_people_indicator,2_against_freedom_indicator,3_sexual_freedom_indicator,4_family_relationships_indicator,5_property_crimes_indicator,6_collective_security_indicator,7_forgeries_indicator,8_public_administration_indicator,9_justice_administration_indicator
0,2019,0,37.3873,-5.9869,8.414.240,1_against_people,37130,1,0,0,0,0,0,0,0,0
1,2019,1,41.6488,-0.8891,1.319.291,1_against_people,4760,1,0,0,0,0,0,0,0,0
2,2019,2,43.3619,-5.8494,1.022.800,1_against_people,3368,1,0,0,0,0,0,0,0,0
3,2019,3,39.5712,2.6466,1.149.460,1_against_people,8110,1,0,0,0,0,0,0,0,0
4,2019,5,28.2916,-16.6291,2.153.389,1_against_people,12396,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,2010,16,42.6954,-1.6761,636.924,9_justice_administration,235,0,0,0,0,0,0,0,0,1
1706,2010,17,43.2630,-2.9349,2.178.339,9_justice_administration,1170,0,0,0,0,0,0,0,0,1
1707,2010,18,42.2871,-2.5396,322.415,9_justice_administration,137,0,0,0,0,0,0,0,0,1
1708,2010,9,35.8894,-5.3198,80.579,9_justice_administration,101,0,0,0,0,0,0,0,0,1


In [942]:
df_values = get_previous_years(df_unpivot_transformed)

df_values 

,year,region,latitude,longitude,population,values_year,1_against_people_indicator,2_against_freedom_indicator,3_sexual_freedom_indicator,4_family_relationships_indicator,5_property_crimes_indicator,6_collective_security_indicator,7_forgeries_indicator,8_public_administration_indicator,9_justice_administration_indicator,values_year_minus_1,values_year_minus_2,values_year_minus_3
0,2019,0,37.3873,-5.9869,8.414.240,37130,1,0,0,0,0,0,0,0,0,35043.0,35087.0,33661.0
1,2019,1,41.6488,-0.8891,1.319.291,4760,1,0,0,0,0,0,0,0,0,4474.0,4146.0,4005.0
2,2019,2,43.3619,-5.8494,1.022.800,3368,1,0,0,0,0,0,0,0,0,3159.0,3160.0,3239.0
3,2019,3,39.5712,2.6466,1.149.460,8110,1,0,0,0,0,0,0,0,0,7545.0,7096.0,6609.0
4,2019,5,28.2916,-16.6291,2.153.389,12396,1,0,0,0,0,0,0,0,0,11805.0,11744.0,11021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,2010,16,42.6954,-1.6761,636.924,235,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN
1706,2010,17,43.2630,-2.9349,2.178.339,1170,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN
1707,2010,18,42.2871,-2.5396,322.415,137,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN
1708,2010,9,35.8894,-5.3198,80.579,101,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN


In [943]:
df_total_values = df_values.dropna(subset=['values_year_minus_1', 'values_year_minus_2', 'values_year_minus_3']).reset_index(drop=True)

df_total_values

,year,region,latitude,longitude,population,values_year,1_against_people_indicator,2_against_freedom_indicator,3_sexual_freedom_indicator,4_family_relationships_indicator,5_property_crimes_indicator,6_collective_security_indicator,7_forgeries_indicator,8_public_administration_indicator,9_justice_administration_indicator,values_year_minus_1,values_year_minus_2,values_year_minus_3
0,2019,0,37.3873,-5.9869,8.414.240,37130,1,0,0,0,0,0,0,0,0,35043.0,35087.0,33661.0
1,2019,1,41.6488,-0.8891,1.319.291,4760,1,0,0,0,0,0,0,0,0,4474.0,4146.0,4005.0
2,2019,2,43.3619,-5.8494,1.022.800,3368,1,0,0,0,0,0,0,0,0,3159.0,3160.0,3239.0
3,2019,3,39.5712,2.6466,1.149.460,8110,1,0,0,0,0,0,0,0,0,7545.0,7096.0,6609.0
4,2019,5,28.2916,-16.6291,2.153.389,12396,1,0,0,0,0,0,0,0,0,11805.0,11744.0,11021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,2013,16,42.6954,-1.6761,644.477,198,0,0,0,0,0,0,0,0,1,234.0,230.0,235.0
1193,2013,17,43.2630,-2.9349,2.191.682,1470,0,0,0,0,0,0,0,0,1,1458.0,1357.0,1170.0
1194,2013,18,42.2871,-2.5396,322.027,115,0,0,0,0,0,0,0,0,1,147.0,141.0,137.0
1195,2013,9,35.8894,-5.3198,84.180,125,0,0,0,0,0,0,0,0,1,157.0,84.0,101.0


In [944]:
# Tabla final

#df_total_values.to_csv('df_total_values.csv', index=False)

In [945]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
columns_pca = ['values_year_minus_1', 'values_year_minus_2', 'values_year_minus_3']
values_year_data = df_total_values[columns_pca]
scaler = StandardScaler()
values_year_scaled = scaler.fit_transform(values_year_data)
pca = PCA(n_components=0.9, random_state=42)
pca_result = pca.fit_transform(values_year_scaled)

df_total_values["value_year_PCA"] = pca_result
df_total_values.drop(columns=['values_year_minus_1', 'values_year_minus_2', 'values_year_minus_3'], inplace=True)


df_total_values["population"] = df_total_values["population"].replace("\.", "", regex=True).astype(int)

<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:14: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Iván\AppData\Local\Temp\ipykernel_1992\1645690850.py:14: SyntaxWarning: invalid escape sequence '\.'
  df_total_values["population"] = df_total_values["population"].replace("\.", "", regex=True).astype(int)


In [946]:
df_total_values_201017 = df_total_values[df_total_values['year'] <= 2017]

df_total_values_2018 = df_total_values[df_total_values['year'] == 2018]

In [947]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 12));
# sns.heatmap(df_total_values_201017.corr(), annot=True, cmap='YlGnBu', linewidths=0.5)           

<Figure size 1500x1200 with 0 Axes>

In [948]:
# from sklearn.model_selection import train_test_split

# features = df_total_values_201017.drop(columns=['values_year'])
# target = df_total_values_201017['values_year']

# X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=42)  
# 
# PARA SERIES DE TIEMPO NO SE USA ESTO, SE FILTRA TODO LO ANTERIOR A 2017 COMO TRAIN Y 2017 COMO TEST
# LUEGO CAMBIO VALORES REALES DE 2017 POR PREDECIDOS Y PREDIGO 2018 Y LUEGO CON 2019
# TRAS ESTO SACO EL MAE Y RMSE Y EL R2 SCORE PARA VER CUAL ES EL MEJOR MODELO
# 
# EL MEJOR MODELO LE PASO LOS DATOS REALES HASTA 2019 PARA PREDECIR 2020, 2021 Y 2022    

In [949]:
X_train = df_total_values_201017.drop(columns=['values_year'])
y_train = df_total_values_201017['values_year']

X_test = df_total_values_2018.drop(columns=['values_year'])
y_test = df_total_values_2018['values_year']

In [950]:
# from sklearn.ensemble import BaggingRegressor
# from sklearn.tree import DecisionTreeRegressor

# bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
#                                n_estimators=100,
#                                max_samples = 500)

# bagging_reg.fit(X_train, y_train)

In [951]:
# from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error

# pred = bagging_reg.predict(X_test)

# print("MAE", mean_absolute_error(pred, y_test))
# print("RMSE", root_mean_squared_error(pred, y_test))
# print("R2 score", bagging_reg.score(X_test, y_test))

In [952]:
#2. NORMALIZACIÓN DE DATOS

from sklearn.preprocessing import MinMaxScaler

normalizer = MinMaxScaler()

X_train_norm = normalizer.fit_transform(X_train)        

X_test_norm = normalizer.transform(X_test)

In [953]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)        # Vuelvo a dar formato de df
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)

In [954]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

bagging_reg_norm = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 500)

bagging_reg_norm.fit(X_train_norm, y_train)

BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=20), max_samples=500,
                 n_estimators=100)

In [955]:
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error

pred_norm = bagging_reg_norm.predict(X_test_norm)

print("MAE", mean_absolute_error(pred_norm, y_test))
print("RMSE", root_mean_squared_error(pred_norm, y_test))
print("R2 score", bagging_reg_norm.score(X_test_norm, y_test))

MAE 974.5877777777779
RMSE 4852.318259434619
R2 score 0.9890246334397367


In [956]:
# 3. Con scaling

scaler = StandardScaler()       
X_train_scaling = scaler.fit_transform(X_train)
X_train_scaling = pd.DataFrame(X_train_scaling, columns = X_train.columns)

X_test_scaling = scaler.transform(X_test)
X_test_scaling = pd.DataFrame(X_test_scaling, columns = X_test.columns)

In [957]:
bagging_reg_scal= BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 500)

bagging_reg_scal.fit(X_train_scaling, y_train)

BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=20), max_samples=500,
                 n_estimators=100)

In [958]:
pred_scal = bagging_reg_scal.predict(X_test_scaling)

print("MAE", mean_absolute_error(pred_scal, y_test))
print("RMSE", root_mean_squared_error(pred_scal, y_test))
print("R2 score", bagging_reg_scal.score(X_test_scaling, y_test))

MAE 1050.479584795322
RMSE 4883.209104658795
R2 score 0.9888844457912511


In [959]:
# import joblib
# joblib.dump(normalizer, "normalizer.pkl")

In [960]:
pred_norm

array([3.5336790e+04, 3.5406500e+03, 3.0325200e+03, 6.7946700e+03,
       1.0245290e+04, 2.2191800e+03, 6.8685400e+03, 6.2239300e+03,
       1.9505570e+04, 1.9686230e+04, 3.0227900e+03, 7.3852500e+03,
       2.2945190e+04, 5.9780200e+03, 2.4908000e+03, 6.0807800e+03,
       9.6782000e+02, 6.8167000e+02, 8.9139000e+02, 2.0355390e+04,
       4.1923500e+03, 2.1911000e+03, 5.6716700e+03, 9.7990600e+03,
       1.6992800e+03, 5.5716500e+03, 5.6373900e+03, 1.9056780e+04,
       1.2569780e+04, 4.1775800e+03, 6.9523900e+03, 1.3990900e+04,
       4.6164900e+03, 2.1971200e+03, 9.1054400e+03, 5.5353000e+02,
       2.9251000e+02, 3.4125000e+02, 2.2044900e+03, 2.8006000e+02,
       1.5639000e+02, 5.2532000e+02, 6.0333000e+02, 7.1340000e+01,
       3.5185000e+02, 3.5777000e+02, 2.0976000e+03, 1.4289200e+03,
       1.6155000e+02, 4.8242000e+02, 1.6561800e+03, 3.9668000e+02,
       2.4335000e+02, 4.5627000e+02, 4.7714000e+01, 2.8640000e+01,
       3.5120000e+01, 1.6114700e+03, 1.6839000e+02, 1.2929000e

In [961]:
X_test["prediction"] = pred_norm

X_test["real"] = y_test

X_test

,year,region,latitude,longitude,population,1_against_people_indicator,2_against_freedom_indicator,3_sexual_freedom_indicator,4_family_relationships_indicator,5_property_crimes_indicator,6_collective_security_indicator,7_forgeries_indicator,8_public_administration_indicator,9_justice_administration_indicator,value_year_PCA,prediction,real
19,2018,0,37.3873,-5.9869,8384408,1,0,0,0,0,0,0,0,0,1.018908,35336.790,35043
20,2018,1,41.6488,-0.8891,1308728,1,0,0,0,0,0,0,0,0,-0.302771,3540.650,4474
21,2018,2,43.3619,-5.8494,1028244,1,0,0,0,0,0,0,0,0,-0.331669,3032.520,3159
22,2018,3,39.5712,2.6466,1128908,1,0,0,0,0,0,0,0,0,-0.192667,6794.670,7545
23,2018,5,28.2916,-16.6291,2127685,1,0,0,0,0,0,0,0,0,0.022440,10245.290,11805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,2018,16,42.6954,-1.6761,647554,0,0,0,0,0,0,0,0,1,-0.455145,314.810,407
1098,2018,17,43.2630,-2.9349,2199088,0,0,0,0,0,0,0,0,1,-0.411588,1448.475,1502
1099,2018,18,42.2871,-2.5396,315675,0,0,0,0,0,0,0,0,1,-0.462450,154.220,185
1100,2018,9,35.8894,-5.3198,85144,0,0,0,0,0,0,0,0,1,-0.461895,155.880,178


In [962]:
df_nuevo = df_total_values.copy()

df_nuevo

,year,region,latitude,longitude,population,values_year,1_against_people_indicator,2_against_freedom_indicator,3_sexual_freedom_indicator,4_family_relationships_indicator,5_property_crimes_indicator,6_collective_security_indicator,7_forgeries_indicator,8_public_administration_indicator,9_justice_administration_indicator,value_year_PCA
0,2019,0,37.3873,-5.9869,8414240,37130,1,0,0,0,0,0,0,0,0,0.862743
1,2019,1,41.6488,-0.8891,1319291,4760,1,0,0,0,0,0,0,0,0,-0.305943
2,2019,2,43.3619,-5.8494,1022800,3368,1,0,0,0,0,0,0,0,0,-0.345329
3,2019,3,39.5712,2.6466,1149460,8110,1,0,0,0,0,0,0,0,0,-0.195319
4,2019,5,28.2916,-16.6291,2153389,12396,1,0,0,0,0,0,0,0,0,-0.024596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,2013,16,42.6954,-1.6761,644477,198,0,0,0,0,0,0,0,0,1,-0.458865
1193,2013,17,43.2630,-2.9349,2191682,1470,0,0,0,0,0,0,0,0,1,-0.416703
1194,2013,18,42.2871,-2.5396,322027,115,0,0,0,0,0,0,0,0,1,-0.462375
1195,2013,9,35.8894,-5.3198,84180,125,0,0,0,0,0,0,0,0,1,-0.463435


In [963]:
df_total_values_2018 = df_total_values[df_total_values['year'] == 2018]

X_train_201017 = df_total_values_201017.drop(columns=['values_year'])
X_test_2018 = df_total_values_2018.drop(columns=['values_year'])

y_train_201017 = df_total_values_201017['values_year']
y_test_2018 = df_total_values_2018['values_year']



In [964]:
X_train_norm_201017 = normalizer.fit_transform(X_train_201017)        

X_test_norm_18 = normalizer.transform(X_test_2018)

In [965]:
X_train_norm_201017 = pd.DataFrame(X_train_norm_201017, columns = X_train_norm_201017.columns)        # Vuelvo a dar formato de df
X_test_2018 = pd.DataFrame(X_test_2018, columns = X_test_2018.columns)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [ ]:
bagging_reg_norm_2018 = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 500)

bagging_reg_norm_2018.fit(X_train_norm_201017, y_train_201017)

In [ ]:
pred_norm_2018 = bagging_reg_norm_2018.predict(X_test_2018)

print("MAE", mean_absolute_error(pred_norm_2018, y_test_2018))
print("RMSE", root_mean_squared_error(pred_norm_2018, y_test_2018))
print("R2 score", bagging_reg_norm.score(X_test_norm_18, y_test_2018))